In [236]:
import numpy as np
import pandas as pd
import re
import requests
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
from bs4 import BeautifulSoup

In [237]:
df = pd.read_csv(r"/content/drive/MyDrive/ev_database.csv")
df

,product_name,Availability,Price,Vechile_range,Efficiency,Weight,Acceleration,Long_distance,battery,Fast_Charge,...,Uk_Price,Driver_Type,No.of_Seats,Heat_Pump,Towbar,Rating,Bidirectional,Load,Home,Grid
0,MG MG4 Electric 64 kWh,Available to order since October 2022,€103 /km€275 /km,360 km,171 Wh/km,1726 kg,7.9 sec,405 km,61.7 kWh,115 kW,...,"£29,745",Rear Wheel Drive,5,1,1,5.0,1,1,1,1
1,Volvo EX60 P12 AWD,Available to order since January 2026,€120 /km€275 /km,610 km,184 Wh/km,2405 kg,3.9 sec,742 km,112.0 kWh,250 kW,...,"£64,860",All Wheel Drive,5,1,1,NaN,1,1,1,1
2,BMW iX3 50 xDrive,Available to order since September 2025,€114 /km€275 /km,610 km,178 Wh/km,2360 kg,4.9 sec,742 km,108.7 kWh,230 kW,...,"£58,755",All Wheel Drive,5,1,1,NaN,1,1,1,1
3,BMW iX xDrive40,Discontinued (October 2021 - January 2025),€225 /km€275 /km,360 km,197 Wh/km,2440 kg,6.1 sec,388 km,71.0 kWh,102 kW,...,"£69,905",All Wheel Drive,5,1,1,5.0,0,1,1,1
4,Fiat 500e Hatchback 42 kWh,Available to order since November 2020,€128 /km€275 /km,235 km,159 Wh/km,1365 kg,9.0 sec,261 km,37.3 kWh,67 kW,...,"£23,995",Front Wheel Drive,4,0,0,4.0,0,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1209,Citroen e-SpaceTourer XS 50 kWh,Discontinued (November 2020 - October 2021),€275 /km€275 /km,185 km,250 Wh/km,1932 kg,12.1 sec,191 km,46.3 kWh,78 kW,...,NaN,Front Wheel Drive,7,0,1,NaN,0,1,1,1
1210,Citroen e-SpaceTourer XL 50 kWh,Discontinued (November 2020 - January 2024),€291 /km€275 /km,180 km,257 Wh/km,1989 kg,12.1 sec,191 km,46.3 kWh,78 kW,...,"£37,885",Front Wheel Drive,7,1,1,NaN,0,1,1,1
1211,Fiat E-Ulysse L3 50 kWh,Discontinued (June 2022 - January 2024),€317 /km€275 /km,180 km,257 Wh/km,1989 kg,12.1 sec,191 km,46.3 kWh,78 kW,...,NaN,Front Wheel Drive,7,0,1,NaN,0,1,1,1
1212,Porsche Taycan Turbo Cross Turismo,Discontinued (January 2023 - February 2024),€380 /km€275 /km,425 km,197 Wh/km,2395 kg,3.3 sec,538 km,83.7 kWh,216 kW,...,"£126,800",All Wheel Drive,5,1,0,NaN,0,1,1,1


In [238]:
df.shape

(1214, 24)

In [239]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1214 entries, 0 to 1213
Data columns (total 24 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   product_name       1214 non-null   object 
 1   Availability       1214 non-null   object 
 2   Price              1214 non-null   object 
 3   Vechile_range      1214 non-null   object 
 4   Efficiency         1214 non-null   object 
 5   Weight             1214 non-null   object 
 6   Acceleration       1214 non-null   object 
 7   Long_distance      1214 non-null   object 
 8   battery            1214 non-null   object 
 9   Fast_Charge        1214 non-null   object 
 10  Towing             1214 non-null   object 
 11  Cargo_vol          1214 non-null   object 
 12  German_Price       1137 non-null   object 
 13  Netherlands_Price  1097 non-null   object 
 14  Uk_Price           884 non-null    object 
 15  Driver_Type        1214 non-null   object 
 16  No.of_Seats        1214 

In [240]:
df.describe()

,No.of_Seats,Heat_Pump,Towbar,Rating,Bidirectional,Load,Home,Grid
count,1214.000000,1214.000000,1214.000000,728.000000,1214.000000,1214.0,1214.0,1214.0
mean,5.219110,0.794893,0.699341,4.813187,0.290774,1.0,1.0,1.0
std,0.823058,0.403946,0.458734,0.575274,0.454307,0.0,0.0,0.0
min,2.000000,0.000000,0.000000,0.000000,0.000000,1.0,1.0,1.0
25%,5.000000,1.000000,0.000000,5.000000,0.000000,1.0,1.0,1.0
50%,5.000000,1.000000,1.000000,5.000000,0.000000,1.0,1.0,1.0
75%,5.000000,1.000000,1.000000,5.000000,1.000000,1.0,1.0,1.0
max,7.000000,1.000000,1.000000,5.000000,1.000000,1.0,1.0,1.0


In [241]:
df.describe(include="all")

,product_name,Availability,Price,Vechile_range,Efficiency,Weight,Acceleration,Long_distance,battery,Fast_Charge,...,Uk_Price,Driver_Type,No.of_Seats,Heat_Pump,Towbar,Rating,Bidirectional,Load,Home,Grid
count,1214,1214,1214,1214,1214,1214,1214,1214,1214,1214,...,884,1214,1214.000000,1214.000000,1214.000000,728.000000,1214.000000,1214.0,1214.0,1214.0
unique,860,409,266,109,120,534,121,415,194,103,...,753,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,Tesla Model 3 Long Range AWD,Available to order since July 2025,€124 /km€275 /km,350 km,168 Wh/km,2395 kg,9.0 sec,252 km,77.0 kWh,110 kW,...,"£44,990",All Wheel Drive,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,6,42,22,35,37,10,38,27,65,68,...,9,487,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,5.219110,0.794893,0.699341,4.813187,0.290774,1.0,1.0,1.0
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.823058,0.403946,0.458734,0.575274,0.454307,0.0,0.0,0.0
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,2.000000,0.000000,0.000000,0.000000,0.000000,1.0,1.0,1.0
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,5.000000,1.000000,0.000000,5.000000,0.000000,1.0,1.0,1.0
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,5.000000,1.000000,1.000000,5.000000,0.000000,1.0,1.0,1.0
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,5.000000,1.000000,1.000000,5.000000,1.000000,1.0,1.0,1.0


In [242]:
df.isnull().sum()

,0
product_name,0
Availability,0
Price,0
Vechile_range,0
Efficiency,0
Weight,0
Acceleration,0
Long_distance,0
battery,0
Fast_Charge,0


In [243]:
df.duplicated().sum()

np.int64(0)

In [244]:
df["Rating"].mode()[0]

np.float64(5.0)

In [245]:
df["Rating"].fillna(df["Rating"].mode()[0], inplace =True)

In [246]:
df.isnull().sum()

,0
product_name,0
Availability,0
Price,0
Vechile_range,0
Efficiency,0
Weight,0
Acceleration,0
Long_distance,0
battery,0
Fast_Charge,0


In [247]:
df.duplicated().sum()

np.int64(0)

In [248]:
df["Brand"]=df["product_name"].apply(lambda x: re.findall(r"^\w+", x)[0])

In [249]:
df.head(1)

,product_name,Availability,Price,Vechile_range,Efficiency,Weight,Acceleration,Long_distance,battery,Fast_Charge,...,Driver_Type,No.of_Seats,Heat_Pump,Towbar,Rating,Bidirectional,Load,Home,Grid,Brand
0,MG MG4 Electric 64 kWh,Available to order since October 2022,€103 /km€275 /km,360 km,171 Wh/km,1726 kg,7.9 sec,405 km,61.7 kWh,115 kW,...,Rear Wheel Drive,5,1,1,5.0,1,1,1,1,MG


In [250]:
df["Range"]=df["Vechile_range"].apply(lambda x:re.findall(r"[0-9]+", x)[0])

In [251]:
df.head(1)

,product_name,Availability,Price,Vechile_range,Efficiency,Weight,Acceleration,Long_distance,battery,Fast_Charge,...,No.of_Seats,Heat_Pump,Towbar,Rating,Bidirectional,Load,Home,Grid,Brand,Range
0,MG MG4 Electric 64 kWh,Available to order since October 2022,€103 /km€275 /km,360 km,171 Wh/km,1726 kg,7.9 sec,405 km,61.7 kWh,115 kW,...,5,1,1,5.0,1,1,1,1,MG,360


In [252]:
df["efficiency"]=df["Efficiency"].apply(lambda x:re.findall(r"[0-9]+", x)[0])

In [253]:
df.head(1)

,product_name,Availability,Price,Vechile_range,Efficiency,Weight,Acceleration,Long_distance,battery,Fast_Charge,...,Heat_Pump,Towbar,Rating,Bidirectional,Load,Home,Grid,Brand,Range,efficiency
0,MG MG4 Electric 64 kWh,Available to order since October 2022,€103 /km€275 /km,360 km,171 Wh/km,1726 kg,7.9 sec,405 km,61.7 kWh,115 kW,...,1,1,5.0,1,1,1,1,MG,360,171


In [254]:
df["weight"]=df["Weight"].apply(lambda x:re.findall(r"[0-9]+", x)[0])

In [255]:
df.head(1)

,product_name,Availability,Price,Vechile_range,Efficiency,Weight,Acceleration,Long_distance,battery,Fast_Charge,...,Towbar,Rating,Bidirectional,Load,Home,Grid,Brand,Range,efficiency,weight
0,MG MG4 Electric 64 kWh,Available to order since October 2022,€103 /km€275 /km,360 km,171 Wh/km,1726 kg,7.9 sec,405 km,61.7 kWh,115 kW,...,1,5.0,1,1,1,1,MG,360,171,1726


In [256]:
df["speed"]=df["Acceleration"].apply(lambda x:re.findall(r"[.0-9]+", x)[0])

In [257]:
df.head(1)

,product_name,Availability,Price,Vechile_range,Efficiency,Weight,Acceleration,Long_distance,battery,Fast_Charge,...,Rating,Bidirectional,Load,Home,Grid,Brand,Range,efficiency,weight,speed
0,MG MG4 Electric 64 kWh,Available to order since October 2022,€103 /km€275 /km,360 km,171 Wh/km,1726 kg,7.9 sec,405 km,61.7 kWh,115 kW,...,5.0,1,1,1,1,MG,360,171,1726,7.9


In [258]:
df["Fast_charge"]=df["Fast_Charge"].apply(lambda x:re.findall(r"[0-9]+", x)[0])

In [259]:
df.head(1)

,product_name,Availability,Price,Vechile_range,Efficiency,Weight,Acceleration,Long_distance,battery,Fast_Charge,...,Bidirectional,Load,Home,Grid,Brand,Range,efficiency,weight,speed,Fast_charge
0,MG MG4 Electric 64 kWh,Available to order since October 2022,€103 /km€275 /km,360 km,171 Wh/km,1726 kg,7.9 sec,405 km,61.7 kWh,115 kW,...,1,1,1,1,MG,360,171,1726,7.9,115


In [260]:
df.head(1)

,product_name,Availability,Price,Vechile_range,Efficiency,Weight,Acceleration,Long_distance,battery,Fast_Charge,...,Bidirectional,Load,Home,Grid,Brand,Range,efficiency,weight,speed,Fast_charge
0,MG MG4 Electric 64 kWh,Available to order since October 2022,€103 /km€275 /km,360 km,171 Wh/km,1726 kg,7.9 sec,405 km,61.7 kWh,115 kW,...,1,1,1,1,MG,360,171,1726,7.9,115


In [261]:
df["distance"]=df["Long_distance"].apply(lambda x:re.findall(r"[0-9]+", x)[0])

In [262]:
df.head(1)

,product_name,Availability,Price,Vechile_range,Efficiency,Weight,Acceleration,Long_distance,battery,Fast_Charge,...,Load,Home,Grid,Brand,Range,efficiency,weight,speed,Fast_charge,distance
0,MG MG4 Electric 64 kWh,Available to order since October 2022,€103 /km€275 /km,360 km,171 Wh/km,1726 kg,7.9 sec,405 km,61.7 kWh,115 kW,...,1,1,1,MG,360,171,1726,7.9,115,405


In [263]:
df["cargo_vol"] = df["Cargo_vol"].apply(lambda x: (re.findall(r"[0-9]+", str(x)) or [np.nan])[0] if pd.notna(x) else np.nan)
df["cargo_vol"] = pd.to_numeric(df["cargo_vol"], errors="coerce").astype("Int64")

In [264]:
df.head(1)

,product_name,Availability,Price,Vechile_range,Efficiency,Weight,Acceleration,Long_distance,battery,Fast_Charge,...,Home,Grid,Brand,Range,efficiency,weight,speed,Fast_charge,distance,cargo_vol
0,MG MG4 Electric 64 kWh,Available to order since October 2022,€103 /km€275 /km,360 km,171 Wh/km,1726 kg,7.9 sec,405 km,61.7 kWh,115 kW,...,1,1,MG,360,171,1726,7.9,115,405,363


In [265]:
df["Battery"] = df["battery"].apply(lambda x: (re.findall(r"[0-9.]+", str(x)) or [np.nan])[0] if pd.notna(x) else np.nan)
df["Battery"] = pd.to_numeric(df["Battery"], errors="coerce")

In [266]:
df.head(1)

,product_name,Availability,Price,Vechile_range,Efficiency,Weight,Acceleration,Long_distance,battery,Fast_Charge,...,Grid,Brand,Range,efficiency,weight,speed,Fast_charge,distance,cargo_vol,Battery
0,MG MG4 Electric 64 kWh,Available to order since October 2022,€103 /km€275 /km,360 km,171 Wh/km,1726 kg,7.9 sec,405 km,61.7 kWh,115 kW,...,1,MG,360,171,1726,7.9,115,405,363,61.7


In [267]:
df['Battery'].unique()

array([ 61.7, 112. , 108.7,  71. ,  37.3,  58. ,  75. ,  85. ,  77. ,
        60. ,  45.1,  78. ,  91.3,  79. ,  94. ,  65. ,  52. ,  68. ,
        60.5,  57. ,  64.8,  58.9,  67. ,  86.5,  74. ,  80. ,  65.4,
        64.7,  43.8, 118. ,  80.7,  73.5,  32. ,  96. ,  82.5,  76. ,
        36. ,  72. ,  46. ,  87. ,  64. , 108.8,  51. ,  70.5,  62.1,
        62. ,  59. ,  49. , 117. ,  43.2,  38.3,  76.6,  95. ,  37.9,
        55. ,  94.9,  60.2,  87.5,  68.3,  91. ,  43.6,  27.5,  65.2,
       105.2,  71.8,  81.3, 102. ,  50.8,  66.5,  86. ,  96.9,  28.9,
        40. ,  73. ,  67.1,  81.9,  39. ,  57.4, 106. ,  53.5, 116. ,
        72.5,  28. , 108. ,  39.2,  50. ,  92.2,  46.3, 109.1,  81.2,
        30. ,  32.3,  41.2,  49.2,  52.5,  42.5,  90. ,  16.7,  85.4,
        74.4, 123. ,  24. ,  70. ,  69. ,  67.8,  96.5,  54. , 101.7,
        69.9,  83.7,  25. ,  94.8, 100. ,  74.3,  61.4,  21.3,  59.8,
        61.9,  45. ,  29.8,  36.6,  80.8,  88. ,  37. ,  85.5,  28.5,
        82.7,  98. ,

In [268]:
df["german_price"] = df["German_Price"].apply(lambda x: re.sub(r"[€,]", "", x) if pd.notna(x) else np.nan)
df["german_price"] = pd.to_numeric(df["german_price"], errors="coerce").astype("Int64")

In [269]:
df.dtypes

,0
product_name,object
Availability,object
Price,object
Vechile_range,object
Efficiency,object
Weight,object
Acceleration,object
Long_distance,object
battery,object
Fast_Charge,object


In [270]:
df.head(1)

,product_name,Availability,Price,Vechile_range,Efficiency,Weight,Acceleration,Long_distance,battery,Fast_Charge,...,Brand,Range,efficiency,weight,speed,Fast_charge,distance,cargo_vol,Battery,german_price
0,MG MG4 Electric 64 kWh,Available to order since October 2022,€103 /km€275 /km,360 km,171 Wh/km,1726 kg,7.9 sec,405 km,61.7 kWh,115 kW,...,MG,360,171,1726,7.9,115,405,363,61.7,39990


In [271]:
df["netherland_price"] = df["Netherlands_Price"].apply(lambda x: re.sub(r"[€*,]", "", x) if pd.notna(x) else np.nan)
df["netherland_price"] = pd.to_numeric(df["netherland_price"], errors="coerce").astype("Int64")

In [272]:
df.head(1)

,product_name,Availability,Price,Vechile_range,Efficiency,Weight,Acceleration,Long_distance,battery,Fast_Charge,...,Range,efficiency,weight,speed,Fast_charge,distance,cargo_vol,Battery,german_price,netherland_price
0,MG MG4 Electric 64 kWh,Available to order since October 2022,€103 /km€275 /km,360 km,171 Wh/km,1726 kg,7.9 sec,405 km,61.7 kWh,115 kW,...,360,171,1726,7.9,115,405,363,61.7,39990,37570


In [273]:
df['uk_price']=df["Uk_Price"].apply(lambda x: re.sub(r"[£,]", "", str(x)) if pd.notna(x) else np.nan)
df['uk_price']=pd.to_numeric(df["uk_price"], errors="coerce").astype("Int64")

In [274]:
df.columns

Index(['product_name', 'Availability', 'Price', 'Vechile_range', 'Efficiency',
       'Weight', 'Acceleration', 'Long_distance', 'battery', 'Fast_Charge',
       'Towing', 'Cargo_vol', 'German_Price', 'Netherlands_Price', 'Uk_Price',
       'Driver_Type', 'No.of_Seats', 'Heat_Pump', 'Towbar', 'Rating',
       'Bidirectional', 'Load', 'Home', 'Grid', 'Brand', 'Range', 'efficiency',
       'weight', 'speed', 'Fast_charge', 'distance', 'cargo_vol', 'Battery',
       'german_price', 'netherland_price', 'uk_price'],
      dtype='object')

In [275]:
df.head(5)

,product_name,Availability,Price,Vechile_range,Efficiency,Weight,Acceleration,Long_distance,battery,Fast_Charge,...,efficiency,weight,speed,Fast_charge,distance,cargo_vol,Battery,german_price,netherland_price,uk_price
0,MG MG4 Electric 64 kWh,Available to order since October 2022,€103 /km€275 /km,360 km,171 Wh/km,1726 kg,7.9 sec,405 km,61.7 kWh,115 kW,...,171,1726,7.9,115,405,363,61.7,39990,37570,29745
1,Volvo EX60 P12 AWD,Available to order since January 2026,€120 /km€275 /km,610 km,184 Wh/km,2405 kg,3.9 sec,742 km,112.0 kWh,250 kW,...,184,2405,3.9,250,742,575,112.0,71990,72995,64860
2,BMW iX3 50 xDrive,Available to order since September 2025,€114 /km€275 /km,610 km,178 Wh/km,2360 kg,4.9 sec,742 km,108.7 kWh,230 kW,...,178,2360,4.9,230,742,578,108.7,70900,70501,58755
3,BMW iX xDrive40,Discontinued (October 2021 - January 2025),€225 /km€275 /km,360 km,197 Wh/km,2440 kg,6.1 sec,388 km,71.0 kWh,102 kW,...,197,2440,6.1,102,388,500,71.0,77300,84998,69905
4,Fiat 500e Hatchback 42 kWh,Available to order since November 2020,€128 /km€275 /km,235 km,159 Wh/km,1365 kg,9.0 sec,261 km,37.3 kWh,67 kW,...,159,1365,9.0,67,261,185,37.3,34990,27999,23995


In [276]:
df["driver_type"] = df["Driver_Type"].apply(lambda x: " ".join(re.findall(r"[A-Za-z]+", str(x))[:2]))

In [277]:
df.head(1)

,product_name,Availability,Price,Vechile_range,Efficiency,Weight,Acceleration,Long_distance,battery,Fast_Charge,...,weight,speed,Fast_charge,distance,cargo_vol,Battery,german_price,netherland_price,uk_price,driver_type
0,MG MG4 Electric 64 kWh,Available to order since October 2022,€103 /km€275 /km,360 km,171 Wh/km,1726 kg,7.9 sec,405 km,61.7 kWh,115 kW,...,1726,7.9,115,405,363,61.7,39990,37570,29745,Rear Wheel


In [278]:
df["price_per_km"] = df["Price"].apply(
    lambda x: (re.findall(r"[0-9.]+", str(x)) or [np.nan])[0] if pd.notna(x) else np.nan
)

df["price_per_km"] = pd.to_numeric(df["price_per_km"], errors="coerce")

In [279]:
df.head(1)

,product_name,Availability,Price,Vechile_range,Efficiency,Weight,Acceleration,Long_distance,battery,Fast_Charge,...,speed,Fast_charge,distance,cargo_vol,Battery,german_price,netherland_price,uk_price,driver_type,price_per_km
0,MG MG4 Electric 64 kWh,Available to order since October 2022,€103 /km€275 /km,360 km,171 Wh/km,1726 kg,7.9 sec,405 km,61.7 kWh,115 kW,...,7.9,115,405,363,61.7,39990,37570,29745,Rear Wheel,103


In [280]:
df["availability_status"] = df["Availability"].apply(
    lambda x: (re.findall(r"Available|Discontinued", str(x)) or [np.nan])[0] if pd.notna(x) else np.nan)

In [281]:
df.head(1)

,product_name,Availability,Price,Vechile_range,Efficiency,Weight,Acceleration,Long_distance,battery,Fast_Charge,...,Fast_charge,distance,cargo_vol,Battery,german_price,netherland_price,uk_price,driver_type,price_per_km,availability_status
0,MG MG4 Electric 64 kWh,Available to order since October 2022,€103 /km€275 /km,360 km,171 Wh/km,1726 kg,7.9 sec,405 km,61.7 kWh,115 kW,...,115,405,363,61.7,39990,37570,29745,Rear Wheel,103,Available


In [282]:
df["towing"] = df["Towing"].apply(lambda x: (re.findall(r"[0-9]+", str(x)) or [np.nan])[0] if pd.notna(x) else np.nan)
df["towing"] = pd.to_numeric(df["towing"], errors="coerce").astype("Int64")

In [283]:
df.dtypes

,0
product_name,object
Availability,object
Price,object
Vechile_range,object
Efficiency,object
Weight,object
Acceleration,object
Long_distance,object
battery,object
Fast_Charge,object


In [284]:
df.head(1)

,product_name,Availability,Price,Vechile_range,Efficiency,Weight,Acceleration,Long_distance,battery,Fast_Charge,...,distance,cargo_vol,Battery,german_price,netherland_price,uk_price,driver_type,price_per_km,availability_status,towing
0,MG MG4 Electric 64 kWh,Available to order since October 2022,€103 /km€275 /km,360 km,171 Wh/km,1726 kg,7.9 sec,405 km,61.7 kWh,115 kW,...,405,363,61.7,39990,37570,29745,Rear Wheel,103,Available,500


In [285]:
df['towing'].value_counts()

,count
towing,
0,336
1000,147
750,140
1500,113
1800,84
1600,75
2000,59
1200,53
2250,25


In [286]:
df.T

,0,1,2,3,4,5,6,7,8,9,...,1204,1205,1206,1207,1208,1209,1210,1211,1212,1213
product_name,MG MG4 Electric 64 kWh,Volvo EX60 P12 AWD,BMW iX3 50 xDrive,BMW iX xDrive40,Fiat 500e Hatchback 42 kWh,CUPRA Born 150 kW - 58 kWh,Tesla Model Y Long Range AWD,Mercedes-Benz CLA 250+,Tesla Model Y Long Range AWD,CUPRA Born 170 kW - 77 kWh,...,Ford e-Tourneo Custom L1 210 kW,Toyota Proace Shuttle M 50 kWh,Porsche Taycan 4S Plus Sport Turismo,Opel Zafira Electric XL 50 kWh,Opel Vivaro-e Combi S 50 kWh,Citroen e-SpaceTourer XS 50 kWh,Citroen e-SpaceTourer XL 50 kWh,Fiat E-Ulysse L3 50 kWh,Porsche Taycan Turbo Cross Turismo,Peugeot e-Traveller Compact 50 kWh
Availability,Available to order since October 2022,Available to order since January 2026,Available to order since September 2025,Discontinued (October 2021 - January 2025),Available to order since November 2020,Discontinued (October 2021 - July 2024),Discontinued (February 2025 - October 2025),Available to order since May 2025,Discontinued (February 2022 - February 2025),Discontinued (March 2022 - July 2024),...,Discontinued (November 2023 - December 2025),Discontinued (June 2021 - November 2023),Discontinued (March 2022 - February 2024),Available to order since January 2024,Discontinued (October 2020 - October 2021),Discontinued (November 2020 - October 2021),Discontinued (November 2020 - January 2024),Discontinued (June 2022 - January 2024),Discontinued (January 2023 - February 2024),Discontinued (January 2021 - January 2022)
Price,€103 /km€275 /km,€120 /km€275 /km,€114 /km€275 /km,€225 /km€275 /km,€128 /km€275 /km,€110 /km€275 /km,€122 /km€275 /km,€92 /km€275 /km,€126 /km€275 /km,€103 /km€275 /km,...,€322 /km€275 /km,€318 /km€275 /km,€277 /km€275 /km,€274 /km€275 /km,€268 /km€275 /km,€275 /km€275 /km,€291 /km€275 /km,€317 /km€275 /km,€380 /km€275 /km,€275 /km€275 /km
Vechile_range,360 km,610 km,610 km,360 km,235 km,350 km,455 km,585 km,445 km,450 km,...,235 km,185 km,430 km,180 km,185 km,185 km,180 km,180 km,425 km,185 km
Efficiency,171 Wh/km,184 Wh/km,178 Wh/km,197 Wh/km,159 Wh/km,166 Wh/km,165 Wh/km,145 Wh/km,169 Wh/km,171 Wh/km,...,272 Wh/km,250 Wh/km,195 Wh/km,257 Wh/km,250 Wh/km,250 Wh/km,257 Wh/km,257 Wh/km,197 Wh/km,250 Wh/km
Weight,1726 kg,2405 kg,2360 kg,2440 kg,1365 kg,1811 kg,2072 kg,2055 kg,2054 kg,1946 kg,...,2557 kg,1969 kg,2325 kg,2056 kg,1932 kg,1932 kg,1989 kg,1989 kg,2395 kg,1932 kg
Acceleration,7.9 sec,3.9 sec,4.9 sec,6.1 sec,9.0 sec,7.3 sec,4.8 sec,6.7 sec,5.0 sec,7.0 sec,...,8.0 sec,12.1 sec,4.0 sec,13.3 sec,12.1 sec,12.1 sec,12.1 sec,12.1 sec,3.3 sec,12.1 sec
Long_distance,405 km,742 km,742 km,388 km,261 km,375 km,535 km,763 km,524 km,488 km,...,245 km,198 km,546 km,191 km,191 km,191 km,191 km,191 km,538 km,191 km
battery,61.7 kWh,112.0 kWh,108.7 kWh,71.0 kWh,37.3 kWh,58.0 kWh,75.0 kWh,85.0 kWh,75.0 kWh,77.0 kWh,...,64.0 kWh,46.3 kWh,83.7 kWh,46.3 kWh,46.3 kWh,46.3 kWh,46.3 kWh,46.3 kWh,83.7 kWh,46.3 kWh
Fast_Charge,115 kW,250 kW,230 kW,102 kW,67 kW,82 kW,124 kW,235 kW,124 kW,115 kW,...,105 kW,78 kW,216 kW,78 kW,78 kW,78 kW,78 kW,78 kW,216 kW,78 kW


In [287]:
df.columns

Index(['product_name', 'Availability', 'Price', 'Vechile_range', 'Efficiency',
       'Weight', 'Acceleration', 'Long_distance', 'battery', 'Fast_Charge',
       'Towing', 'Cargo_vol', 'German_Price', 'Netherlands_Price', 'Uk_Price',
       'Driver_Type', 'No.of_Seats', 'Heat_Pump', 'Towbar', 'Rating',
       'Bidirectional', 'Load', 'Home', 'Grid', 'Brand', 'Range', 'efficiency',
       'weight', 'speed', 'Fast_charge', 'distance', 'cargo_vol', 'Battery',
       'german_price', 'netherland_price', 'uk_price', 'driver_type',
       'price_per_km', 'availability_status', 'towing'],
      dtype='object')

In [288]:
df.drop(columns = ['product_name', 'Availability', 'Price', 'Vechile_range', 'Efficiency', 'Weight', 'Acceleration',
       'Long_distance', 'battery', 'Fast_Charge', 'Towing', 'Cargo_vol',
       'German_Price', 'Netherlands_Price', 'Uk_Price', 'Driver_Type'], inplace = True)

In [289]:
df

,No.of_Seats,Heat_Pump,Towbar,Rating,Bidirectional,Load,Home,Grid,Brand,Range,...,distance,cargo_vol,Battery,german_price,netherland_price,uk_price,driver_type,price_per_km,availability_status,towing
0,5,1,1,5.0,1,1,1,1,MG,360,...,405,363,61.7,39990,37570,29745,Rear Wheel,103,Available,500
1,5,1,1,5.0,1,1,1,1,Volvo,610,...,742,575,112.0,71990,72995,64860,All Wheel,120,Available,2400
2,5,1,1,5.0,1,1,1,1,BMW,610,...,742,578,108.7,70900,70501,58755,All Wheel,114,Available,2000
3,5,1,1,5.0,0,1,1,1,BMW,360,...,388,500,71.0,77300,84998,69905,All Wheel,225,Discontinued,2500
4,4,0,0,4.0,0,1,1,1,Fiat,235,...,261,185,37.3,34990,27999,23995,Front Wheel,128,Available,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1209,7,0,1,5.0,0,1,1,1,Citroen,185,...,191,140,46.3,50880,<NA>,<NA>,Front Wheel,275,Discontinued,1000
1210,7,1,1,5.0,0,1,1,1,Citroen,180,...,191,912,46.3,54430,59019,37885,Front Wheel,291,Discontinued,1000
1211,7,0,1,5.0,0,1,1,1,Fiat,180,...,191,800,46.3,56990,<NA>,<NA>,Front Wheel,317,Discontinued,1000
1212,5,1,0,5.0,0,1,1,1,Porsche,425,...,538,489,83.7,165848,173264,126800,All Wheel,380,Discontinued,0


In [290]:
df.isnull().sum()

,0
No.of_Seats,0
Heat_Pump,0
Towbar,0
Rating,0
Bidirectional,0
Load,0
Home,0
Grid,0
Brand,0
Range,0


In [291]:
df["towing"].median()

np.float64(1000.0)

In [292]:
df["towing"] = df["towing"].fillna(df["towing"].median()).astype("int64")

In [293]:
df["cargo_vol"].median()

np.float64(510.0)

In [294]:
df["cargo_vol"] = df["cargo_vol"].fillna(df["cargo_vol"].median()).astype("int64")

In [295]:
df.isnull().sum()

,0
No.of_Seats,0
Heat_Pump,0
Towbar,0
Rating,0
Bidirectional,0
Load,0
Home,0
Grid,0
Brand,0
Range,0


In [296]:
df.duplicated().sum()

np.int64(0)

In [297]:
df = df[
    ['Brand', 'price_per_km', 'availability_status', 'Range', 'efficiency',
       'weight', 'speed', 'Fast_charge', 'distance', 'cargo_vol', 'Battery',
       'german_price', 'netherland_price', 'uk_price', 'towing','driver_type', 'No.of_Seats', 'Heat_Pump', 'Towbar', 'Rating',
       'Bidirectional', 'Load', 'Home', 'Grid']
]

In [298]:
df['german_price']

,german_price
0,39990
1,71990
2,70900
3,77300
4,34990
...,...
1209,50880
1210,54430
1211,56990
1212,165848


In [299]:
df.isnull().sum()

,0
Brand,0
price_per_km,0
availability_status,0
Range,0
efficiency,0
weight,0
speed,0
Fast_charge,0
distance,0
cargo_vol,0


In [302]:
df['german_price'].median()

np.float64(55400.0)

In [303]:
df['netherland_price'].median()

np.float64(54990.0)

In [304]:
df['uk_price'].median()

np.float64(49825.0)

In [309]:
df['german_price'].fillna(df['german_price'].median(),inplace=True)

In [310]:
df['netherland_price'].fillna(df['netherland_price'].median(),inplace=True)

In [311]:
df['uk_price'].fillna(df['uk_price'].median(),inplace=True)

In [312]:
df.isna().sum()

,0
Brand,0
price_per_km,0
availability_status,0
Range,0
efficiency,0
weight,0
speed,0
Fast_charge,0
distance,0
cargo_vol,0


In [313]:
df.to_csv("ev_database_cleaned.csv", index=False)

In [314]:
df = pd.read_csv("ev_database_cleaned.csv")

In [80]:
df

,Brand,price_per_km,availability_status,Range,efficiency,weight,speed,Fast_charge,distance,cargo_vol,...,towing,driver_type,No.of_Seats,Heat_Pump,Towbar,Rating,Bidirectional,Load,Home,Grid
0,MG,103,Available,360,171,1726,7.9,115,405,363,...,500,Rear Wheel,5,1,1,5.0,1,1,1,1
1,Volvo,120,Available,610,184,2405,3.9,250,742,575,...,2400,All Wheel,5,1,1,5.0,1,1,1,1
2,BMW,114,Available,610,178,2360,4.9,230,742,578,...,2000,All Wheel,5,1,1,5.0,1,1,1,1
3,BMW,225,Discontinued,360,197,2440,6.1,102,388,500,...,2500,All Wheel,5,1,1,5.0,0,1,1,1
4,Fiat,128,Available,235,159,1365,9.0,67,261,185,...,0,Front Wheel,4,0,0,4.0,0,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1209,Citroen,275,Discontinued,185,250,1932,12.1,78,191,140,...,1000,Front Wheel,7,0,1,5.0,0,1,1,1
1210,Citroen,291,Discontinued,180,257,1989,12.1,78,191,912,...,1000,Front Wheel,7,1,1,5.0,0,1,1,1
1211,Fiat,317,Discontinued,180,257,1989,12.1,78,191,800,...,1000,Front Wheel,7,0,1,5.0,0,1,1,1
1212,Porsche,380,Discontinued,425,197,2395,3.3,216,538,489,...,0,All Wheel,5,1,0,5.0,0,1,1,1


In [315]:
df.isnull().sum()

,0
Brand,0
price_per_km,0
availability_status,0
Range,0
efficiency,0
weight,0
speed,0
Fast_charge,0
distance,0
cargo_vol,0
